# Projective Preferential Bayesian Optimization: Campor/Cu(111)

<font size="4">In this notebook the user's belief about an optimal configuration can be elicited by using the [Projective Preferential Bayesian Optimization](https://arxiv.org/abs/2002.03113) framework. The test case is the adsorption of a non-symmetric bulky molecule camphor on the flat surface of (111)-plane terminated Cu slab.</font> 

Note: The last elicitation iteration may take while to finish.

#### Import dependencies

In [1]:
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'

In [ ]:
import os
import sys
sys.path.insert(1, os.getcwd()+'/src')
sys.path.insert(1, os.getcwd()+'/camphor_copper')
import time
from datetime import datetime
import numpy as np
from gui import GUI_session, generate_optimal_configuration
from gp_model import GPModel
from ppbo_settings import PPBO_settings
from acquisition import next_query
from jupyter_ui_poll import run_ui_poll_loop
from ipywidgets import VBox
from IPython.display import display, IFrame, HTML, clear_output
from IPython.core.display import display
display(HTML("<style>div.output_scroll { height: 45em; }</style>")) #Make outputwindow larger

## Session configurations

In [ ]:
session_name = 'TEST'

#### Specify the aquisition strategy and the problem setting
Acquisition startegies with unit projections ($\boldsymbol{\xi}$ is an unit vector):
- PCD = preferential coordinate descent
- EI-EXT = finds unit projection that maximizes EI, and x is set to xstar
- EI-EXT-FAST = same as EI-EXT except $d\mathbf{x}$ integral omitted
- EI-VARMAX = same as EI-EXT except $\mathbf{x}$ is chosen to maximize GP variance
- EI-VARMAX-FAST = same as EI-VARMAX except $d\mathbf{x}$ integral omitted

Acquisition startegies with non-unit projections:
- EI = expected improvement by projective preferential query
- EI-FIXEDX = same as EI except $\mathbf{x}$ is fixed to $\textrm{argmax}_{\mathbf{x}}\mu(\mathbf{x})$ (xstar)
- EXT = pure exploitation
- EXR = pure exploration (variance maximization)
- RAND = random 

In [ ]:
acquisition_strategy = 'PCD'

In [ ]:
PPBO_settings = PPBO_settings(D=6,bounds=((-0.5,0.5),(-0.5,0.5),(4,7),(-180,180),(-180,180),(-180,180)),
                              kernel = 'camphor_copper_kernel',theta_initial=[0.001,0.26,0.1],
                              xi_acquisition_function=acquisition_strategy,verbose=False,
                              skip_computations_during_initialization=True)

original scale = a vector in the space ((-0.5,0.5),(-0.5,0.5),(4,7),(-180,180),(-180,180),(-180,180))<br>
GP domain scale = a vector in the space ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1))

#### Querying settings

In [ ]:
NUMBER_OF_QUERIES = 14 # + 6 initial queries
ADAPTIVE_INITIALIZATION = True  #At initilization: immediatly update the coordinate according to the user feedback

#### Set initial queries

In [ ]:
initial_queries_xi = np.array([list(np.eye(6)[i]) for i in range(6)]) #Initial xi:s correspond to unit vectors
if ADAPTIVE_INITIALIZATION:
    initial_queries_x = np.array([[-0.5, -0.5, 5.0, -84.4, 142.8, 2.7],]*6) #1st coordinate does not have relevance
else:
    initial_queries_x = np.array([[-0.5, -0.5, 5.0, -84.4, 142.8, 2.7],]*6)
    #initial_queries_x = np.random.uniform([PPBO_settings.original_bounds[i][0] for i in range(6)], [PPBO_settings.original_bounds[i][1] for i in range(6)], size=(6,6))
print("Number of initial queries is: " + str(len(initial_queries_xi)))

#### Hyperparameter optimization

In [ ]:
OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION = False
OPTIMIZE_HYPERPARAMETERS_AFTER_QUERY_NUMBER = 999

#### Initialize the user session

In [ ]:
should_log = False
if should_log:
    orig_stdout = sys.stdout
    log_file = open('camphor_copper/user_session_log_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.txt', "w")
    sys.stdout = log_file
GUI_ses = GUI_session(PPBO_settings)
results_mustar = []
results_xstar = []
results_time = []

## Knowledge elicitation

In [ ]:
start = time.time() 

### PPBO event loop

In [ ]:
for i in range(len(initial_queries_xi)+NUMBER_OF_QUERIES):
    if i < len(initial_queries_xi):
        print(f'Initialization in progress... ({i+1}/{len(initial_queries_xi)})\r', end="")
        if i==len(initial_queries_xi)-1:
            GP_model_preference.turn_initialization_off()   
    else:
        print(f'Elicitation in progress... ({i+1-len(initial_queries_xi)}/{NUMBER_OF_QUERIES})\r', end="")
        if i+1==len(initial_queries_xi)+NUMBER_OF_QUERIES:
            GP_model_preference.set_last_iteration()
    ''' Projective preferential query '''
    if i < len(initial_queries_xi):
        xi = initial_queries_xi[i].copy()
        if not i==0 and GUI_ses.user_feedback_preference is not None and ADAPTIVE_INITIALIZATION:
            initial_queries_x[i:,:] = GUI_ses.user_feedback_preference
        x = initial_queries_x[i].copy()
        x[xi!=0] = 0
    else:
        xi,x = next_query(PPBO_settings,GP_model_preference,unscale=True)
    GUI_ses.initialize_iteration(x,xi)
    ''' Event loop '''
    view,button,slider = GUI_ses.getMiniGUI()
    app = VBox([view,slider,button])
    def wait_user_input():
        if not GUI_ses.user_feedback_was_given:
            pass
            return None
        app.close()
        GUI_ses.user_feedback_was_given = False
        GUI_ses.save_results()
        return 1       
    display(app)
    query_presented = time.time()
    dt = run_ui_poll_loop(wait_user_input)
    time_spent = time.time() - query_presented
    ''' Create GP model for first time '''
    if i==0:
        GP_model_preference = GPModel(PPBO_settings)
    ''' Update GP model '''
    GP_model_preference.update_feedback_processing_object(np.array(GUI_ses.results_preference))
    GP_model_preference.update_data()
    if i+1==OPTIMIZE_HYPERPARAMETERS_AFTER_QUERY_NUMBER:
        GP_model_preference.update_model(optimize_theta=True)
        print('Hyperparameters: ' + str(GP_model_preference.theta))
    else:
        GP_model_preference.update_model(optimize_theta=OPTIMIZE_HYPERPARAMETERS_AFTER_EACH_ITERATION)
    results_mustar.append(GP_model_preference.mustar)
    results_xstar.append(GP_model_preference.FP.unscale(GP_model_preference.xstar))
    results_time.append(time_spent)
    clear_output(wait=True)
print("Elicitation done!")

In [ ]:
print("Total time: " + str(time.time()-start) + " seconds.")

In [ ]:
xstar = GP_model_preference.xstar

## Save the results

#### Generate html-file corresponding to the user's most preferred configuration

In [ ]:
optimal_molecule_html = generate_optimal_configuration(GP_model_preference.FP.unscale(xstar))

#### Save the user session results

In [ ]:
#Save results to csv-file
print("Saving the user session results...")
user_session_results = GUI_ses.results_preference.copy()
user_session_results_confidence = GUI_ses.results_confidence.copy()
user_session_results['iter_mustar'] = results_mustar
user_session_results['iter_xstar_unscaled'] = results_xstar
user_session_results['time_feedback'] = results_time
user_session_results.to_csv('camphor_copper/' + str(session_name)+'_results_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.csv',index=False)
user_session_results_confidence.to_csv('camphor_copper/' + str(session_name)+'_confidence_results_'+str(datetime.now().strftime("%d-%m-%Y_%H-%M-%S"))+'.csv',index=False)
#Close the log-file
if should_log:
    sys.stdout = orig_stdout
    log_file.close()

## Analyze the results

#### View the user's most preferred configuration
<font color='red'>Press "i" to restore the default view</font>

In [ ]:
IFrame(src="./camphor_copper/"+str(optimal_molecule_html), width=900, height=500)

#### Slice plots of the utility function (predictive mean)

In [ ]:
import plot_results as pr

In [ ]:
pr.sliceplot_pred_mean('alpha','beta',GP_model_preference,xstar)

In [ ]:
pr.sliceplot_pred_mean('x','y',GP_model_preference,xstar)

In [ ]:
pr.sliceplot_pred_mean('z','gamma',GP_model_preference,xstar)

In [ ]:
print("The most preferred configuration (original scale): " + str(list(GP_model_preference.FP.unscale(xstar))))

In [ ]:
print("The most preferred configuration (GP domain scale): " + str(list(xstar)))

#### The experiment results (for each iteration: preferred molecule and user feedback time)

In [ ]:
display(HTML(user_session_results.iloc[:,14:].to_html()))